# IE 306  SYSTEM SIMULATION
## FURKAN KESKIN - 2018400150
## SINEM KOCOGLU - 2020400339

In [36]:
import numpy as np
from queue import Queue, PriorityQueue

SEED = 2018400150 + 2020400339

nurses = 4  # S
beds = 6    # K
p_1 = 0.25
lambd = 1
mu_T = 0.3125
mu_s = 0.16
mu_cb = 0.1666666667

## PART 1.2

In [62]:
iterations = 50

def Generate_Interarrival():
    return np.random.exponential(1/lambd, iterations)

def Generate_Nurse_Service_Time():
    return np.random.exponential(1/mu_T, iterations)

def Generate_Hospital_Healing_Time():
    return np.random.exponential(1/mu_cb, iterations)

def Generate_Home_Healing_Time(condition):
    if condition == 's':
        return np.random.exponential(1/mu_s, iterations)
    elif condition == 'c':
        alfa = np.random.uniform(1.25, 1.75)
        return np.random.exponential(alfa/mu_cb, iterations)
    
def Generate_Conditions():
    condition_probs = np.random.uniform(0, 1, iterations)
    return np.array(['s' if i <= p_1 else 'c' for i in condition_probs])

In [64]:
np.random.seed(SEED)

interarrivals = Generate_Interarrival()
nurse_service_times = Generate_Nurse_Service_Time()
hospital_healing_times = Generate_Hospital_Healing_Time()
home_healing_times_stable = Generate_Home_Healing_Time('s')
home_healing_times_critical = Generate_Home_Healing_Time('c')
arrivals = np.roll(interarrivals.cumsum(), 1)
conditions = Generate_Conditions()

print(interarrivals)
print(arrivals)
print(nurse_service_times)
print(conditions)

[2.41070487e-01 6.47407769e-01 4.50411894e+00 1.07434257e+00
 2.53375056e+00 3.19182038e-01 1.10184572e+00 5.85289474e-01
 2.28080274e-01 8.26462025e-02 7.52756332e-01 3.82814961e-01
 8.08808754e-01 6.04292502e-02 1.84175342e+00 1.53742158e+00
 3.46427665e-01 1.53435066e-02 2.19875851e-01 4.21094336e+00
 1.47260307e+00 3.26912281e-01 7.63602801e-02 6.63089981e-01
 8.45308865e-02 7.50748687e-01 9.69871693e-02 1.84875141e-01
 2.58576118e+00 8.51753397e-01 6.79433616e-01 7.58412151e-01
 2.45503843e-03 3.42977880e+00 1.72511183e+00 2.11595757e-01
 5.41296407e-01 1.41057061e+00 6.48464237e-01 1.34652378e+00
 2.32358741e-01 7.08265163e-01 1.40769545e+00 3.70437850e+00
 1.54393276e-01 3.57141439e-01 7.35832352e-02 4.76161819e-01
 1.48732732e+00 5.73864356e-02]
[48.00026441  0.24107049  0.88847826  5.3925972   6.46693977  9.00069033
  9.31987237 10.42171809 11.00700757 11.23508784 11.31773404 12.07049038
 12.45330534 13.26211409 13.32254334 15.16429676 16.70171834 17.04814601
 17.06348951 17.2

In [65]:
def Arrival():
    global busy_nurses, nurse_idx
    
    # adding next arrival to the FEL
    FEL.put((arrivals[pid+1], 'A', pid+1))
    
    if busy_nurses < nurses:
        FEL.put((simulation_time + nurse_service_times[nurse_idx], 'DN', pid))
        busy_nurses += 1
        nurse_idx += 1
    else:
        WQ.put((simulation_time, pid))
            
    

def Departure_Triage():
    global busy_nurses, occupied_beds, nurse_idx, condition_idx, hospital_idx, home_stable_idx, home_critical_idx
    
    # should be independent from the SEED
    # condition = 's' if np.random.uniform(0, 1) <= p_1 else 'c'
    condition = conditions[condition_idx]
    condition_idx += 1
    
    # departure of the current patient from the nurse triage
    if condition == 's':
        FEL.put((simulation_time + home_healing_times_stable[home_stable_idx], 'H', pid))
        home_stable_idx += 1
    elif condition == 'c':
        if occupied_beds < beds:
            FEL.put((simulation_time + hospital_healing_times[hospital_idx], 'DB', pid))
            hospital_idx += 1
            occupied_beds += 1
        else:
            FEL.put((simulation_time + home_healing_times_critical[home_critical_idx], 'H', pid))
            home_critical_idx += 1
            
    # arrangements for the FEL
    if WQ.empty():
        busy_nurses -= 1
    else:
        # arrival time queue'da ne kadar beklediğiyle alakalı
        arrival_time_of_incoming, pid_of_incoming = WQ.get()
        FEL.put((simulation_time + nurse_service_times[nurse_idx], 'DN', pid_of_incoming))
        nurse_idx += 1
        
        

def Treated_at_Hospital():
    global occupied_beds, healed_patients
    occupied_beds -= 1
    healed_patients += 1
    

def Healed_at_Home():
    global healed_patients
    healed_patients += 1
    
    
def Advance_Time():
    global simulation_time, event_type, pid, event_no
    simulation_time, event_type, pid = FEL.get()
    event_no += 1
    
    
    
def Execute_Event():
    if event_type == 'A':
        Arrival()
    elif event_type == 'DN':
        Departure_Triage()
    elif event_type == 'DB':
        Treated_at_Hospital()
    elif event_type == 'H':
        Healed_at_Home()
        
    print(event_no, round(simulation_time, 3), event_type, pid, FEL.queue[0], FEL.qsize(), WQ.qsize(), busy_nurses, occupied_beds, healed_patients)
        
    Advance_Time()
 

## PART 2.2

In [72]:
FEL = PriorityQueue()    # Future Event List  
WQ = Queue()             # Waiting Queue for Nurse Service

simulation_time = 0
event_type = 'A'
pid = 0
event_no = 1

busy_nurses = 0          
occupied_beds = 0

nurse_idx = 0
condition_idx = 0
hospital_idx = 0
home_stable_idx = 0
home_critical_idx = 0

healed_patients = 0

# 0, 1, 2 .... idx-1

# System starts with a patient with ID = 0 in simulation time = 0
# t, event, ID


while event_no <= 50:
    Execute_Event()
    


1 0 A 0 (0.0005671254166299415, 'DN', 0) 2 0 1 0 0
2 0.001 DN 0 (0.2410704873911317, 'A', 1) 2 0 0 1 0
3 0.241 A 1 (0.4892989282053244, 'DN', 1) 3 0 1 1 0
4 0.489 DN 1 (0.8884782567780792, 'A', 2) 3 0 0 1 0
5 0.888 A 2 (1.2809798984277663, 'DB', 0) 4 0 1 1 0
6 1.281 DB 0 (1.4326135079781541, 'DN', 2) 3 0 1 0 1
7 1.433 DN 2 (5.392597195791888, 'A', 3) 3 0 0 1 1
8 5.393 A 3 (6.466939768343855, 'A', 4) 4 0 1 1 1
9 6.467 A 4 (7.723892110302097, 'DB', 2) 5 0 2 1 1
10 7.724 DB 2 (8.14672296416439, 'H', 1) 4 0 2 0 2
11 8.147 H 1 (9.000690331158372, 'A', 5) 3 0 2 0 3
12 9.001 A 5 (9.319872369193263, 'A', 6) 4 0 3 0 3
13 9.32 A 6 (9.572673994325278, 'DN', 6) 5 0 4 0 3
14 9.573 DN 6 (9.614286330127127, 'H', 6) 5 0 3 0 3
15 9.614 H 6 (9.779329181922671, 'DN', 3) 4 0 3 0 4
16 9.779 DN 3 (10.421718094088671, 'A', 7) 4 0 2 1 4
17 10.422 A 7 (10.96264712479617, 'DN', 7) 5 0 3 1 4
18 10.963 DN 7 (11.00700756778249, 'A', 8) 5 0 2 2 4
19 11.007 A 8 (11.223645373939192, 'DN', 5) 6 0 3 2 4
20 11.224 DN 5 